In [1]:
# Create Graph From file SNN_metric
import create_snnGraph as cs
import networkx as nx
import sparsify_snnGraph as ss

# a = cs.readData('../snn_computation/snn_metric.csv')
# a = cs.readData('../../SNN-density/snn_computation/snn_metric_ds2.csv')
a = cs.readData('../../SNN-density/snn_computation/snn_metric_ds3.csv')
G = cs.createGraph(a)

print("G edges: ",G.number_of_edges())
print("G nodes: ",G.number_of_nodes())




G edges:  124450
G nodes:  9498


In [2]:
# Silhoutte Coefficient
import pandas as pd
import numpy as np
from os import path
import sys
sys.path.append('../../')
from Evaluation.silhouette_coefficient import silhoutte_result
def read_snnArray(file_path=None):
    df = pd.read_csv(file_path,index_col=False,header=None,sep=' ')
    snn_nparray = np.array(df)
    return snn_nparray
# snn_nparray = read_snnArray('../snn_computation/snn_distance_metric.csv')
snn_nparray2 = read_snnArray('../../SNN-density/snn_computation/snn_distance_metric_ds3.csv')


In [3]:
def demo(G:nx.Graph,distance_metrix,threshold,n):
    G1 =  G.copy()
    snn_nparray = distance_metrix
    label_dictionary = {}
    for j in range (G.number_of_nodes()):
        label_dictionary[j]=-1
    # Sparsify Graph
    g = ss.sparsifyGraph(G1,threshold=threshold)
    sub_graphs = nx.connected_components(g)

    # Create labels for all nodes ,  label -1 for outlier
    i = 0
    for sub in sub_graphs:
        if(len(sub)>n):
            # print(i)
            # print(sub)
            for node in sub:
                label_dictionary[node]=i 
            i +=1
    labels = list(label_dictionary.values())

# Remove outliers

    l = np.array(labels)
    count = 0
    for item in l:
        if item>-1:
            count = count+1
    new_distance_metric = np.zeros((count,count),dtype=float)
    new_distance_metric.shape
    dictionary_no_outliers = {i:label_dictionary[i] for i in label_dictionary if label_dictionary[i]!=-1}

    k = dictionary_no_outliers.keys()
    
    # Create a new name for each node after remove outliers
    change_name_list={}
    start = 0
    for item in k:
        change_name_list[start] = item
        start = start+1

    #Add value to new snn_array, to use for calculate  Silhoutte - after remove outliers
    i=0
    j=0
    for i in range(0,count):
        for j in range(i+1,count):
            new_distance_metric[i][j] = snn_nparray[change_name_list[i]][change_name_list[j]]
            new_distance_metric[j][i] = new_distance_metric[i][j]
     # Silhoutte 
    new_label = list(dictionary_no_outliers.values())
    number_of_cluster = len(set(new_label)) 
    if len(set(new_label)) > 1:
        silhoute_removeouliers= silhoutte_result(new_distance_metric,new_label)
    else:
        print("Number of labels needs to > 1. ERROR!!")
        silhoute_removeouliers = -1

    return silhoute_removeouliers,len(new_label),number_of_cluster,label_dictionary

    

In [4]:
threshold_list = [i for i in range (1,50)]
for item in threshold_list:
    x1,x2,x3,x4 = demo(G,snn_nparray2,item,1)
    print('Threshold: ',item,'score:',x1,'points:', x2,'clusters:', x3)
# print(x2)
# print(x3)

Threshold:  1 score: 0.00025288726176571173 points: 9431 clusters: 16
Threshold:  2 score: 0.0007017305306636791 points: 9307 clusters: 52
Threshold:  3 score: 0.0015203117541109567 points: 8930 clusters: 87
Threshold:  4 score: 0.021437848776648605 points: 7441 clusters: 809
Threshold:  5 score: 0.06385603716084443 points: 5286 clusters: 1327
Threshold:  6 score: 0.11330858195586262 points: 3412 clusters: 1187
Threshold:  7 score: 0.16965004003471004 points: 2128 clusters: 855
Threshold:  8 score: 0.22526728663672282 points: 1437 clusters: 613
Threshold:  9 score: 0.28882393393749456 points: 1026 clusters: 452
Threshold:  10 score: 0.3479264453175878 points: 776 clusters: 349
Threshold:  11 score: 0.3981843388597246 points: 634 clusters: 289
Threshold:  12 score: 0.42673199355937963 points: 565 clusters: 259
Threshold:  13 score: 0.4488162992865269 points: 518 clusters: 239
Threshold:  14 score: 0.4673107326708206 points: 480 clusters: 222
Threshold:  15 score: 0.484636583005609 point

In [5]:
score,number_of_nodes,num_clusters,label_dictionary = demo(G,snn_nparray2,45,1)
print(score,number_of_nodes,num_clusters)
from collections import defaultdict
df = pd.read_csv('../../../TwitterDataset/DE/DE_target.csv')
df = df[['new_id','days','views']]
v = defaultdict(list)

for key, value in sorted(label_dictionary.items()):
    v[value].append(key)
for key in v.keys():
    if (key!=-1):
        print("Cluster: ",key)
        nodes = v[key]
        print(df.loc[df['new_id'].isin(nodes)])

0.9520000000000003 20 10
Cluster:  0
      new_id  days  views
541     1192  2066  22812
4556    1189  2066  23459
Cluster:  1
      new_id  days  views
5236    1227  1546  21032
8922    1563  1546  22543
Cluster:  2
      new_id  days  views
1029    1670   744   2734
2711    1715   744   2630
Cluster:  3
      new_id  days  views
2825    7908  1117    235
9013    2893  1117    104
Cluster:  4
      new_id  days  views
68      8616  1359   1021
6144    3113  1359   1184
Cluster:  5
      new_id  days  views
4526    8500   959   1521
5148    3876   959   1535
Cluster:  6
      new_id  days  views
787     4062  1161   4973
3603    4190  1161   4836
Cluster:  7
      new_id  days  views
5080    6218  1515   4046
5347    4860  1515   4013
Cluster:  8
      new_id  days  views
1129    7636  1229   6408
4616    5354  1229   6312
Cluster:  9
      new_id  days  views
4046    6394   723  12255
7008    6369   723  15023
